<a href="https://colab.research.google.com/github/BaDestin14/RHCheck/blob/main/Analyse_de_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.8 MB/s eta 0:00:00


In [2]:
!pip install pytesseract

In [3]:
import re
import spacy
from pdfminer.high_level import extract_text
import pytesseract
from PIL import Image
from spacy.matcher import Matcher
import pandas as pd
import json

In [4]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 21.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
# Initialiser le modèle SpaCy pour le français
nlp = spacy.load("fr_core_news_sm")

In [6]:
# Configuration de l'OCR (Tesseract)
pytesseract.pytesseract.tesseract_cmd = r'<CHEMIN_VERS_TESSERACT_EXE>'

def extract_data(file_path):
    """Phase 1: Extraction des données (supporte PDF et images)"""
    try:
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            img = Image.open(file_path)
            text = pytesseract.image_to_string(img, lang='fra')
        elif file_path.lower().endswith('.pdf'):
            text = extract_text(file_path)
        else:
            raise ValueError("Format de fichier non supporté")
        return text
    except Exception as e:
        print(f"Erreur d'extraction: {str(e)}")
        return ""

def clean_data(text):
    """Phase 2: Nettoyage et normalisation des données"""
    # Suppression des caractères spéciaux
    text = re.sub(r'[^\w\s.,;:!?À-ÿ-]', ' ', text)
    # Normalisation des espaces
    text = re.sub(r'\s+', ' ', text)
    # Conversion en minuscules
    return text.lower().strip()

def extract_entities(text):
    """Phase 3: Reconnaissance d'entités nommées"""
    doc = nlp(text)
    entities = {
        "PERSON": [],
        "ORG": [],
        "DATE": [],
        "LOC": [],
        "DIPLOME": [],
        "COMPETENCE": []
    }

    # Entités standard
    for ent in doc.ents:
        if ent.label_ in entities and ent.text not in entities[ent.label_]:
            entities[ent.label_].append(ent.text)

    # Règles personnalisées pour diplômes et compétences
    diplome_patterns = [
        [{"LOWER": {"IN": ["bac", "bts", "dut", "licence", "master", "doctorat"]}}],
        [{"LOWER": "bachelor"}, {"LOWER": "of"}, {"LOWER": "science"}],
        [{"LOWER": "ingénieur"}]
    ]

    matcher = Matcher(nlp.vocab)
    matcher.add("DIPLOME", diplome_patterns)

    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        if span.text not in entities["DIPLOME"]:
            entities["DIPLOME"].append(span.text)

    return entities

def extract_relations(entities, text):
    """Phase 4: Extraction des relations entre entités"""
    relations = []
    doc = nlp(text)

    # Exemple: Relation entre compétence et expérience
    for sentence in doc.sents:
        if "expérience" in sentence.text:
            for competence in entities["COMPETENCE"]:
                if competence in sentence.text:
                    relations.append({
                        "relation": "a_experience_avec",
                        "entite": competence,
                        "contexte": sentence.text.strip()
                    })

    return relations

def build_model(entities, relations):
    """Phase 5: Construction du modèle de données"""
    model = {
        "candidat": {
            "nom": entities["PERSON"][0] if entities["PERSON"] else "Inconnu",
            "contact": "",
            "education": entities["DIPLOME"],
            "competences": entities["COMPETENCE"],
            "experiences": []
        },
        "relations": relations
    }

    # Extraction des expériences professionnelles
    for org in entities["ORG"]:
        model["candidat"]["experiences"].append({
            "organisation": org,
            "poste": "À déterminer",
            "duree": ""
        })

    return model

def display_results(model):
    """Phase 6: Présentation des résultats"""
    print("\n" + "="*50)
    print("ANALYSE COMPLÈTE DU CV".center(50))
    print("="*50)

    print(f"\nCandidat: {model['candidat']['nom']}")
    print("\nFormation:")
    for i, diplome in enumerate(model['candidat']['education'], 1):
        print(f"  {i}. {diplome}")

    print("\nCompétences techniques:")
    for i, competence in enumerate(model['candidat']['competences'], 1):
        print(f"  {i}. {competence}")

    print("\nExpériences professionnelles:")
    for i, exp in enumerate(model['candidat']['experiences'], 1):
        print(f"  {i}. {exp['organisation']} - {exp['poste']}")

    print("\n" + "="*50)
    print("Relations identifiées:".center(50))
    for rel in model['relations']:
        print(f"- {rel['entite']} ({rel['relation']})")

    # Export JSON
    with open('cv_analysis.json', 'w') as f:
        json.dump(model, f, indent=2, ensure_ascii=False)

    return model


In [7]:
# Pipeline complet
def analyze_cv(file_path):
    print(f"Début de l'analyse du fichier: {file_path}")

    # Phase 1: Extraction
    raw_text = extract_data(file_path)
    if not raw_text:
        return None

    # Phase 2: Nettoyage
    cleaned_text = clean_data(raw_text)

    # Phase 3: Entités
    entities = extract_entities(cleaned_text)

    # Phase 4: Relations
    relations = extract_relations(entities, cleaned_text)

    # Phase 5: Modélisation
    model = build_model(entities, relations)

    # Phase 6: Présentation
    return display_results(model)

# Exemple d'utilisation
if __name__ == "__main__":
    cv_path = "chemin/vers/votre/cv.pdf"  # ou .jpg/.png
    result = analyze_cv(cv_path)

    if result:
        print("\n>>> Analyse terminée avec succès!")
        print(f">>> Données exportées dans 'cv_analysis.json'")

Début de l'analyse du fichier: chemin/vers/votre/cv.pdf
Erreur d'extraction: [Errno 2] No such file or directory: 'chemin/vers/votre/cv.pdf'


###1. Identifier et supprimer le bruit et les informations non pertinentes
Le "bruit" dans le texte peut inclure des caractères spéciaux, des espaces multiples, des sauts de ligne, etc. Les expressions régulières (avec la bibliothèque re) sont très efficaces pour cela.

**Exemples de nettoyage de texte :**

In [8]:
import re

text_with_noise = "  Ceci est un   texte avec... des caractères spéciaux!!! et des\n sauts de ligne.  "

# Supprimer les caractères spéciaux (tout ce qui n'est pas une lettre, un chiffre ou un espace)
cleaned_text = re.sub(r'[^\w\s]', '', text_with_noise)

# Remplacer les espaces multiples par un seul espace
cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

# Supprimer les espaces au début et à la fin
cleaned_text = cleaned_text.strip()

print(f"Texte original: '{text_with_noise}'")
print(f"Texte nettoyé: '{cleaned_text}'")

Texte original: '  Ceci est un   texte avec... des caractères spéciaux!!! et des
 sauts de ligne.  '
Texte nettoyé: 'Ceci est un texte avec des caractères spéciaux et des sauts de ligne'


###2. Gérer les incohérences de formatage
Les dates, les numéros de téléphone, les adresses, etc., peuvent avoir des formats très variés.

**Exemple avec les dates :**

La bibliothèque dateutil est excellente pour analyser différents formats de date.

In [9]:
from dateutil import parser

date_strings = ["2023-10-26", "26/10/2023", "October 26, 2023", "26 Oct 2023"]

for date_str in date_strings:
    try:
        parsed_date = parser.parse(date_str)
        print(f"'{date_str}' -> {parsed_date.strftime('%Y-%m-%d')}")
    except ValueError:
        print(f"Impossible de parser '{date_str}'")

'2023-10-26' -> 2023-10-26
'26/10/2023' -> 2023-10-26
'October 26, 2023' -> 2023-10-26
'26 Oct 2023' -> 2023-10-26


###3. Techniques de normalisation : Tokenisation, Stemming et Lemmatisation
Ces techniques permettent de ramener les mots à une forme de base, ce qui est essentiel pour de nombreuses tâches de NLP.

* **Tokenisation :** Diviser le texte en mots ou en phrases (tokens).
* **Stemming (Racinisation) :** Réduire les mots à leur racine (parfois la racine n'est pas un mot réel). C'est plus rapide mais moins précis.
* **Lemmatisation :** Réduire les mots à leur forme de dictionnaire (lemme). C'est plus précis mais plus lent.
Exemple avec NLTK (une autre bibliothèque NLP populaire) :

In [14]:
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import zipfile
import urllib.request

# Define the directory for NLTK data
nltk_data_dir = '/content/nltk_data'
os.makedirs(nltk_data_dir, exist_ok=True)
nltk.data.path = [nltk_data_dir]

# --- Manual Download and Unzip ---
# Define URLs and target paths
punkt_url = 'https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/tokenizers/punkt.zip'
wordnet_url = 'https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/wordnet.zip'
punkt_zip_path = os.path.join(nltk_data_dir, 'punkt.zip')
wordnet_zip_path = os.path.join(nltk_data_dir, 'wordnet.zip')
punkt_dir = os.path.join(nltk_data_dir, 'tokenizers')
wordnet_dir = os.path.join(nltk_data_dir, 'corpora')

# Download and unzip Punkt if not present
if not os.path.exists(os.path.join(punkt_dir, 'punkt')):
    print("Downloading and unzipping Punkt...")
    os.makedirs(punkt_dir, exist_ok=True)
    urllib.request.urlretrieve(punkt_url, punkt_zip_path)
    with zipfile.ZipFile(punkt_zip_path, 'r') as zip_ref:
        zip_ref.extractall(nltk_data_dir)
    os.remove(punkt_zip_path)
    print("Punkt downloaded.")

# Download and unzip WordNet if not present
if not os.path.exists(os.path.join(wordnet_dir, 'wordnet')):
    print("Downloading and unzipping WordNet...")
    os.makedirs(wordnet_dir, exist_ok=True)
    urllib.request.urlretrieve(wordnet_url, wordnet_zip_path)
    with zipfile.ZipFile(wordnet_zip_path, 'r') as zip_ref:
        zip_ref.extractall(nltk_data_dir)
    os.remove(wordnet_zip_path)
    print("WordNet downloaded.")


# --- Original Code ---
text_to_normalize = "running ran runs easily"

# Tokenisation
tokens = word_tokenize(text_to_normalize)
print(f"Tokens: {tokens}")

# Stemming
stemmer = PorterStemmer()
stemmed_tokens = [stemmer.stem(token) for token in tokens]
print(f"Stemmed: {stemmed_tokens}")

# Lemmatisation
lemmatizer = WordNetLemmatizer()
lemmatized_tokens = [lemmatizer.lemmatize(token, pos='v') for token in tokens] # 'v' pour verbe
print(f"Lemmatized: {lemmatized_tokens}")

WordNet downloaded.
Tokens: ['running', 'ran', 'runs', 'easily']
Stemmed: ['run', 'ran', 'run', 'easili']
Lemmatized: ['run', 'run', 'run', 'easily']


## Set up the environment

### Subtask:
Install necessary libraries for Streamlit, database interaction, and additional ML models.


**Reasoning**:
The previous command failed because `sqlite3` is a built-in Python library and does not need to be installed via pip. The other packages, `streamlit` and `scikit-learn`, should still be installed.



In [15]:
!pip install streamlit scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.7 MB/s eta 0:00:00


## Research and select additional ml models

### Subtask:
Identify suitable ML models for CV classification (e.g., classifying candidates by role, experience level, etc.).

**Reasoning**:
Research and identify suitable ML models for CV classification based on common approaches for text classification, considering their applicability to CV data.

In [16]:
# 1. Research common machine learning approaches for text classification,
#    specifically applied to CVs or resumes.

# Common approaches for text classification:
# - Traditional ML: Naive Bayes, SVM, Logistic Regression, Decision Trees, Random Forests
# - Feature Extraction for Traditional ML: Bag-of-Words (BoW), TF-IDF, Word Embeddings (used with pooling)
# - Deep Learning: Recurrent Neural Networks (RNNs like LSTMs, GRUs), Convolutional Neural Networks (CNNs),
#                  Transformer-based models (BERT, RoBERTa, etc.)
# - Feature Extraction for Deep Learning: Word Embeddings (Word2Vec, GloVe, FastText), Contextual Embeddings (BERT, etc.)

# Applications to CVs:
# - Classifying job roles (e.g., Software Engineer, Data Scientist, Marketing Manager)
# - Classifying experience levels (e.g., Junior, Mid-level, Senior)
# - Classifying industry
# - Classifying required skills/qualifications

# 2. Identify at least two different types of models that could be suitable for classifying CVs.

# Model Type 1: Traditional ML (e.g., SVM with TF-IDF features)
# Reasoning:
# - Strengths: Relatively simple to understand and implement, often perform well on text classification tasks with appropriate feature engineering (like TF-IDF), computationally less expensive than deep learning models for training.
# - Applicability to CVs: Can effectively capture the importance of specific terms (skills, job titles, degree names) using TF-IDF, which are highly relevant for CV classification. SVMs are known for their effectiveness in high-dimensional spaces like text data.

# Model Type 2: Deep Learning (e.g., Transformer-based model like BERT)
# Reasoning:
# - Strengths: Capable of capturing complex patterns, context, and semantic relationships in text data, which is crucial for understanding the nuances in a CV. Pre-trained transformer models (like BERT) can leverage vast amounts of external text data, requiring less domain-specific data for fine-tuning.
# - Applicability to CVs: Can understand the context in which skills, experiences, and education are presented, potentially leading to more accurate classification than models relying solely on word frequency (TF-IDF). Can handle variations in phrasing and structure.

# 3. Briefly explain the reasoning behind selecting these models.
# (Explanation is included in the reasoning section above for each model type)

# 4. Do not implement or train the models at this stage, only research and select.
# (This step is adhered to)

print("Selected ML Models for CV Classification:")
print("- Traditional ML: Support Vector Machine (SVM) with TF-IDF features")
print("  Reasoning: Good performance on text classification, captures term importance.")
print("- Deep Learning: Transformer-based model (e.g., BERT)")
print("  Reasoning: Captures context and semantic relationships, leverages pre-training on large text corpora.")

Selected ML Models for CV Classification:
- Traditional ML: Support Vector Machine (SVM) with TF-IDF features
  Reasoning: Good performance on text classification, captures term importance.
- Deep Learning: Transformer-based model (e.g., BERT)
  Reasoning: Captures context and semantic relationships, leverages pre-training on large text corpora.


## Train and evaluate ml models

### Subtask:
Prepare a dataset, train the selected ML models, and evaluate their performance.

**Reasoning**:
The subtask requires preparing a dataset, training an SVM model with TF-IDF features, and evaluating its performance. This involves loading/creating data, preprocessing text, vectorizing using TF-IDF, splitting data, training SVM, and evaluating. I will combine these steps into a single code block. Since a dataset is not provided, I will create a synthetic one for demonstration purposes.

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd

# 1. Load or create a dataset (creating a synthetic one)
data = {
    'cv_text': [
        "Experienced software engineer with skills in Python, Java, and SQL. Worked at Google for 5 years. Master's degree in Computer Science.",
        "Marketing manager with expertise in digital marketing, social media, and content creation. 3 years experience at Facebook. Bachelor's degree in Marketing.",
        "Data scientist with strong background in machine learning, statistics, and R. PhD in Statistics from Stanford.",
        "Junior software developer with knowledge of C++ and algorithms. Internship at Microsoft. Pursuing Bachelor's degree.",
        "Senior marketing specialist with focus on SEO and analytics. 10 years experience in the industry. MBA.",
        "Recent graduate with a Bachelor's in Computer Science. Skills: Python, HTML, CSS. Looking for entry-level software engineering roles.",
        "Experienced project manager with PMP certification. Managed large teams at Amazon. Background in engineering.",
        "Graphic designer with proficiency in Adobe Creative Suite. Portfolio available. Bachelor's degree in Fine Arts.",
        "Business analyst with experience in data analysis and strategic planning. Worked in consulting for 4 years. Master's in Business Administration.",
        "Entry-level data analyst with skills in Excel and SQL. Internship experience. Bachelor's degree in Economics."
    ],
    'job_role': [
        'Software Engineer', 'Marketing', 'Data Scientist', 'Software Engineer', 'Marketing',
        'Software Engineer', 'Project Manager', 'Graphic Designer', 'Business Analyst', 'Data Analyst'
    ],
    'experience_level': [
        'Senior', 'Mid-level', 'Senior', 'Junior', 'Senior',
        'Junior', 'Senior', 'Entry-level', 'Mid-level', 'Entry-level'
    ]
}
df = pd.DataFrame(data)

# For this subtask, we will classify by 'job_role'
X = df['cv_text']
y = df['job_role']

# 2. Preprocess text data (tokenization, lowercase, remove stop words)
# TF-IDF Vectorizer handles tokenization, lowercase, and can remove stop words
tfidf_vectorizer = TfidfVectorizer(stop_words='english', lowercase=True)

# 3. Convert preprocessed text data into numerical features using TF-IDF vectorization.
X_tfidf = tfidf_vectorizer.fit_transform(X)

# 4. Split the dataset into training and testing sets.
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

# 5. Initialize and train the SVM model on the training data using the TF-IDF features.
svm_model = SVC(kernel='linear') # Using linear kernel for simplicity and often good performance on text
svm_model.fit(X_train_tfidf, y_train)

# 6. Evaluate the performance of the trained SVM model on the testing data.
y_pred_svm = svm_model.predict(X_test_tfidf)

print("SVM Model Performance (TF-IDF Features):")
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_svm))

# Note: The dataset is very small and synthetic, so performance metrics will not be representative of real-world scenarios.
# This serves as a demonstration of the steps.

SVM Model Performance (TF-IDF Features):
Accuracy: 0.3333333333333333

Classification Report:
                   precision    recall  f1-score   support

 Business Analyst       0.00      0.00      0.00         1
        Marketing       0.00      0.00      0.00         1
Software Engineer       0.33      1.00      0.50         1

         accuracy                           0.33         3
        macro avg       0.11      0.33      0.17         3
     weighted avg       0.11      0.33      0.17         3



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Reasoning**:
The previous step completed the SVM model training and evaluation. The next steps in the subtask involve working with a Deep Learning model (BERT). This requires installing necessary libraries, preprocessing data using a BERT tokenizer, preparing the data for BERT, splitting the data, loading a pre-trained BERT model, fine-tuning it, and evaluating its performance. I will combine the installation and the BERT tokenization and data preparation steps into this code block. Since we need to fine-tune BERT, we'll use the `transformers` library from Hugging Face and `torch` or `tensorflow` for the deep learning framework. I will use `torch`.

In [18]:
!pip install transformers[torch]
!pip install accelerate -U

from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.functional import cross_entropy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd # Re-import just in case, though likely available

# Assuming df, X, y are already defined from the previous step

# 7. Preprocess the text data by tokenizing it using a BERT tokenizer.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Map labels to integers
label_map = {label: i for i, label in enumerate(y.unique())}
y_encoded = y.map(label_map)

# 8. Prepare the data in a format suitable for training a BERT model
class CVDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length', # Pad to max_len
            truncation=True, # Truncate if longer than max_len
            return_attention_mask=True,
            return_tensors='pt', # Return PyTorch tensors
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Define max sequence length (adjust based on typical CV length)
MAX_LEN = 256 # Example length

# 9. Split the dataset into training and testing sets.
# Removed stratify due to small dataset size and classes with only one member
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Create datasets
train_dataset = CVDataset(
    texts=X_train.tolist(),
    labels=y_train_encoded.tolist(),
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

test_dataset = CVDataset(
    texts=X_test.tolist(),
    labels=y_test_encoded.tolist(),
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

print(f"Training dataset size: {len(train_dataset)}")
print(f"Testing dataset size: {len(test_dataset)}")
print(f"Number of unique labels: {len(label_map)}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Training dataset size: 7
Testing dataset size: 3
Number of unique labels: 7


**Reasoning**:
Continue with the "Train and evaluate ML models" subtask. The previous step prepared the data for the BERT model. This step involves loading a pre-trained `BertForSequenceClassification` model, defining the training loop, training the model, and evaluating its performance on the test set.

In [19]:
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification
from torch.optim import AdamW
from tqdm.notebook import tqdm # Using tqdm.notebook for Colab progress bars

# 10. Load a pre-trained BERT model for sequence classification.
# Specify the number of output labels (number of unique job roles)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map))

# Set device for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Create data loaders
BATCH_SIZE = 8 # Adjust based on available memory
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# 11. Define the training parameters (optimizer, loss function, number of epochs).
optimizer = AdamW(model.parameters(), lr=2e-5) # Standard learning rate for fine-tuning BERT
# Loss function is typically handled internally by BertForSequenceClassification,
# but we can explicitly use CrossEntropyLoss if needed.

EPOCHS = 3 # Number of training epochs (adjust as needed)

# 12. Implement the training loop to fine-tune the BERT model on the training data.
print(f"Training BERT model on {device}...")
model.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        model.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Clip gradients to prevent exploding gradients
        optimizer.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1} - Average Training Loss: {avg_train_loss:.4f}")

# 13. Evaluate the performance of the trained BERT model on the testing data.
print("\nEvaluating BERT model...")
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Evaluating"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)

        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).flatten()

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Convert predictions back to original labels
id_to_label = {i: label for label, i in label_map.items()}
predicted_labels = [id_to_label[p] for p in predictions]
true_original_labels = [id_to_label[t] for t in true_labels]


print("\nBERT Model Performance:")
print("Accuracy:", accuracy_score(true_original_labels, predicted_labels))
print("\nClassification Report:")
# Handle potential errors with classification_report on small datasets with missing labels in prediction
try:
    print(classification_report(true_original_labels, predicted_labels, zero_division=0))
except ValueError as e:
    print(f"Could not generate detailed classification report due to missing labels in predictions: {e}")
    # Print a simpler report if detailed one fails
    from sklearn.metrics import precision_score, recall_score, f1_score
    print(f"Precision: {precision_score(true_original_labels, predicted_labels, average='weighted', zero_division=0):.4f}")
    print(f"Recall: {recall_score(true_original_labels, predicted_labels, average='weighted', zero_division=0):.4f}")
    print(f"F1-score: {f1_score(true_original_labels, predicted_labels, average='weighted', zero_division=0):.4f}")


# Note: Performance on this small synthetic dataset will not be indicative of real-world performance.
# This code demonstrates the process of fine-tuning and evaluating a BERT model for text classification.

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training BERT model on cpu...


Epoch 1:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1 - Average Training Loss: 2.0911


...

Epoch 2 - Average Training Loss: 2.0739


Epoch 3:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 3 - Average Training Loss: 1.7760

Evaluating BERT model...


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]


BERT Model Performance:
Accuracy: 0.3333333333333333

Classification Report:
                   precision    recall  f1-score   support

 Business Analyst       0.00      0.00      0.00         1
        Marketing       0.00      0.00      0.00         1
Software Engineer       0.33      1.00      0.50         1

         accuracy                           0.33         3
        macro avg       0.11      0.33      0.17         3
     weighted avg       0.11      0.33      0.17         3



## Integrate ML models

### Subtask:
Incorporate the trained ML models into the CV analysis pipeline to perform classification.

**Reasoning**:
Modify the `analyze_cv` function to include loading the trained ML models and using them to predict the job role and/or experience level based on the extracted text. Since the models trained on the synthetic data are not practical for real-world use, this implementation will focus on the structural integration, potentially using placeholder logic or assuming pre-trained models are available.

In [21]:
# Assuming the trained models (svm_model, tfidf_vectorizer for SVM and model, tokenizer for BERT)
# are available or can be loaded here. For demonstration with synthetic data,
# we'll use the models trained in previous steps or add placeholder logic.

# It's more practical in a real application to save and load models.
# Example of saving/loading (commented out):
# import joblib # for SVM and TF-IDF vectorizer
# joblib.dump(svm_model, 'svm_model.pkl')
# joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')
# svm_model_loaded = joblib.load('svm_model.pkl')
# tfidf_vectorizer_loaded = joblib.load('tfidf_vectorizer.pkl')

# For BERT, saving/loading is done using Hugging Face methods:
# model.save_pretrained('./bert_cv_classifier')
# tokenizer.save_pretrained('./bert_cv_classifier')
# bert_model_loaded = BertForSequenceClassification.from_pretrained('./bert_cv_classifier')
# bert_tokenizer_loaded = BertTokenizer.from_pretrained('./bert_cv_classifier')


# Placeholder function for classifying using ML models
def classify_cv(text, svm_model, tfidf_vectorizer, bert_model, tokenizer, label_map):
    """Classifies the CV text using trained ML models."""
    # --- SVM Classification ---
    # Preprocess and vectorize the input text
    text_tfidf = tfidf_vectorizer.transform([text])
    # Predict using the SVM model
    svm_prediction_encoded = svm_model.predict(text_tfidf)[0]
    # Convert prediction back to original label
    id_to_label = {i: label for label, i in label_map.items()}
    svm_predicted_label = id_to_label.get(svm_prediction_encoded, "Unknown")

    # --- BERT Classification ---
    # Preprocess the input text using BERT tokenizer
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=MAX_LEN, # Use the same MAX_LEN as during training
        return_token_type_ids=True,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    # Move model to evaluation mode and perform prediction
    bert_model.eval()
    with torch.no_grad():
        input_ids = encoding['input_ids'].to(device) # Assuming 'device' is defined (cuda or cpu)
        attention_mask = encoding['attention_mask'].to(device)
        outputs = bert_model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        bert_prediction_encoded = torch.argmax(logits, dim=1).flatten().item()

    # Convert prediction back to original label
    bert_predicted_label = id_to_label.get(bert_prediction_encoded, "Unknown")

    return {
        "job_role_svm": svm_predicted_label,
        "job_role_bert": bert_predicted_label
        # Could add experience level classification here too
    }


# Modify the analyze_cv function to include classification
def analyze_cv(file_path, svm_model, tfidf_vectorizer, bert_model, tokenizer, label_map):
    print(f"Début de l'analyse du fichier: {file_path}")

    # Phase 1: Extraction
    raw_text = extract_data(file_path)
    if not raw_text:
        return None

    # Phase 2: Nettoyage
    cleaned_text = clean_data(raw_text)

    # Phase 3: Entités
    entities = extract_entities(cleaned_text)

    # Phase 4: Relations
    relations = extract_relations(entities, cleaned_text)

    # Phase 5: Modélisation
    model_data = build_model(entities, relations)

    # Phase 6: Classification using ML models
    # Pass the cleaned text to the classification function
    classification_results = classify_cv(cleaned_text, svm_model, tfidf_vectorizer, bert_model, tokenizer, label_map)
    # Add classification results to the model data
    model_data['candidat'].update(classification_results)


    # Phase 7: Save to database
    save_analysis_to_db(model_data)

    # Phase 8: Présentation
    return display_results(model_data)

# Example usage (assuming models are loaded/available in the environment)
# In a real application, you would load your trained models here before calling analyze_cv
# For this demonstration, we'll assume the models from the previous training steps are still in memory.
# You would also need a real CV file path here.
# cv_path = "chemin/vers/votre/cv.pdf"
# result = analyze_cv(cv_path, svm_model, tfidf_vectorizer, model, tokenizer, label_map) # Use 'model' as bert_model variable name

# if result:
#     print("\n>>> Analyse terminée avec succès!")
#     print(f">>> Données exportées dans 'cv_analysis.json' et sauvegardées dans '{DATABASE_NAME}'")
#     print("\n>>> Classification Results:")
#     print(f"    Job Role (SVM): {result['candidat'].get('job_role_svm', 'N/A')}")
#     print(f"    Job Role (BERT): {result['candidat'].get('job_role_bert', 'N/A')}")

In [22]:
import streamlit as st
import os
import tempfile
import json

# Re-import necessary functions from the previous cells
# Ensure the previous cells defining these functions (extract_data, clean_data,
# extract_entities, extract_relations, build_model, display_results, analyze_cv)
# have been executed.

# Configuration de l'OCR (Tesseract) - needs to be set if not already
# pytesseract.pytesseract.tesseract_cmd = r'<CHEMIN_VERS_TESSERACT_EXE>' # Replace with your Tesseract path

st.title("Analyse de CV")

uploaded_file = st.file_uploader("Choisissez un fichier PDF ou image (PNG, JPG, JPEG)", type=['pdf', 'png', 'jpg', 'jpeg'])

if uploaded_file is not None:
    # To use the existing analyze_cv function, we need a file path.
    # Streamlit's file_uploader provides an in-memory file object.
    # We can save it to a temporary file to get a path.
    with tempfile.NamedTemporaryFile(delete=False, suffix=os.path.splitext(uploaded_file.name)[1]) as tmp_file:
        tmp_file.write(uploaded_file.getvalue())
        tmp_file_path = tmp_file.name

    st.info(f"Fichier téléchargé: {uploaded_file.name}")

    # Call the existing analysis pipeline
    st.text("Analyse en cours...")
    analysis_result = analyze_cv(tmp_file_path)

    # Clean up the temporary file
    os.unlink(tmp_file_path)

    if analysis_result:
        st.success("Analyse terminée avec succès!")

        st.subheader("Résultats de l'analyse")

        st.markdown("#### Informations Candidat")
        st.write(f"**Nom:** {analysis_result['candidat']['nom']}")
        # Add other basic info like contact if extracted later

        st.markdown("#### Formation")
        if analysis_result['candidat']['education']:
            for i, diplome in enumerate(analysis_result['candidat']['education'], 1):
                st.write(f"- {diplome}")
        else:
            st.write("Aucune information de formation trouvée.")

        st.markdown("#### Compétences")
        if analysis_result['candidat']['competences']:
            for i, competence in enumerate(analysis_result['candidat']['competences'], 1):
                 st.write(f"- {competence}")
        else:
            st.write("Aucune compétence trouvée.")

        st.markdown("#### Expériences Professionnelles")
        if analysis_result['candidat']['experiences']:
            for i, exp in enumerate(analysis_result['candidat']['experiences'], 1):
                st.write(f"- **Organisation:** {exp['organisation']}")
                # Add poste, duree if extracted later
        else:
            st.write("Aucune expérience professionnelle trouvée.")

        st.markdown("#### Relations identifiées")
        if analysis_result['relations']:
            for rel in analysis_result['relations']:
                 st.write(f"- {rel['entite']} ({rel['relation']})")
        else:
            st.write("Aucune relation identifiée.")


    else:
        st.error("Échec de l'analyse du CV. Veuillez vérifier le fichier.")


2025-07-25 03:00:13.891 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 03:00:14.564 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-07-25 03:00:14.566 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 03:00:14.568 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 03:00:14.570 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 03:00:14.572 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 03:00:14.577 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 03:00:14.581 Thread 'MainThread': mi

# Task
Extend the existing CV analysis system with a web front-end (Streamlit), a database for storing results, and additional ML models for classification.

## Integrate streamlit

### Subtask:
Create a basic Streamlit application to upload files and display initial results from the existing analysis functions.


**Reasoning**:
Implement the basic Streamlit application structure with file upload and initial result display by incorporating the existing analysis functions.



## Database setup

### Subtask:
Choose and set up a database (e.g., SQLite for simplicity, or PostgreSQL/MySQL for scalability). Define the schema for storing CV analysis results.

**Reasoning**:
The subtask requires preparing a dataset, training an SVM model with TF-IDF features, and evaluating its performance. This involves loading/creating data, preprocessing text, vectorizing using TF-IDF, splitting data, training SVM, and evaluating. I will combine these steps into a single code block. Since a dataset is not provided, I will create a synthetic one for demonstration purposes.



**Reasoning**:
Define the database schema and write the Python code to connect to an SQLite database and create the necessary tables.

In [23]:
import sqlite3

DATABASE_NAME = 'cv_analysis.db'

def create_database_and_tables():
    """Connects to SQLite DB and creates tables if they don't exist."""
    conn = None
    try:
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()

        # Create tables
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS candidates (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                contact TEXT
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS education (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                candidate_id INTEGER,
                degree TEXT NOT NULL,
                FOREIGN KEY (candidate_id) REFERENCES candidates(id)
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS skills (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                candidate_id INTEGER,
                skill TEXT NOT NULL,
                FOREIGN KEY (candidate_id) REFERENCES candidates(id)
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS experiences (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                candidate_id INTEGER,
                organization TEXT NOT NULL,
                position TEXT,
                duration TEXT,
                FOREIGN KEY (candidate_id) REFERENCES candidates(id)
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS relations (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                candidate_id INTEGER,
                relation_type TEXT,
                entity TEXT,
                context TEXT,
                FOREIGN KEY (candidate_id) REFERENCES candidates(id)
            )
        ''')

        conn.commit()
        print(f"Database '{DATABASE_NAME}' and tables created successfully or already exist.")

    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        if conn:
            conn.close()

# Execute the function to create the database and tables
create_database_and_tables()

Database 'cv_analysis.db' and tables created successfully or already exist.


## Integrate database

### Subtask:
Modify the Python code to store the analysis results (extracted entities, relations, etc.) into the database after processing a CV.

**Reasoning**:
Import the sqlite3 library and define the `save_analysis_to_db` function to store the analysis results in the database. This function will handle connecting to the database, inserting data into the respective tables, and handling potential errors.

In [24]:
import sqlite3

DATABASE_NAME = 'cv_analysis.db'

def save_analysis_to_db(model):
    """Saves the CV analysis results into the SQLite database."""
    conn = None
    try:
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()

        # Insert candidate information
        candidate_name = model['candidat']['nom']
        candidate_contact = model['candidat']['contact'] # Assuming contact will be added later
        cursor.execute("INSERT INTO candidates (name, contact) VALUES (?, ?)", (candidate_name, candidate_contact))
        candidate_id = cursor.lastrowid

        # Insert education
        for degree in model['candidat']['education']:
            cursor.execute("INSERT INTO education (candidate_id, degree) VALUES (?, ?)", (candidate_id, degree))

        # Insert skills
        for skill in model['candidat']['competences']:
            cursor.execute("INSERT INTO skills (candidate_id, skill) VALUES (?, ?)", (candidate_id, skill))

        # Insert experiences
        for exp in model['candidat']['experiences']:
            organization = exp.get('organisation', 'N/A')
            position = exp.get('poste', 'À déterminer')
            duration = exp.get('duree', '')
            cursor.execute("INSERT INTO experiences (candidate_id, organization, position, duration) VALUES (?, ?, ?, ?)",
                           (candidate_id, organization, position, duration))

        # Insert relations
        for rel in model['relations']:
            relation_type = rel.get('relation', 'N/A')
            entity = rel.get('entite', 'N/A')
            context = rel.get('contexte', '')
            cursor.execute("INSERT INTO relations (candidate_id, relation_type, entity, context) VALUES (?, ?, ?, ?)",
                           (candidate_id, relation_type, entity, context))

        conn.commit()
        print("Analysis results saved to database.")

    except sqlite3.Error as e:
        print(f"Database error while saving analysis: {e}")
        if conn:
            conn.rollback() # Rollback changes if an error occurs
    finally:
        if conn:
            conn.close()

## Implement database interaction in streamlit

### Subtask:
Add functionality to the Streamlit app to retrieve and display analysis results from the database.

**Reasoning**:
Import necessary libraries for the Streamlit app and database interaction, and define the function to retrieve analysis results from the database.

In [27]:
import streamlit as st
import os
import tempfile
import json
import sqlite3

# Assuming DATABASE_NAME is defined in a previous cell
DATABASE_NAME = 'cv_analysis.db' # Ensure DATABASE_NAME is defined

def create_database_and_tables():
    """Connects to SQLite DB and creates tables if they don't exist."""
    conn = None
    try:
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()

        # Create tables
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS candidates (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                contact TEXT
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS education (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                candidate_id INTEGER,
                degree TEXT NOT NULL,
                FOREIGN KEY (candidate_id) REFERENCES candidates(id)
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS skills (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                candidate_id INTEGER,
                skill TEXT NOT NULL,
                FOREIGN KEY (candidate_id) REFERENCES candidates(id)
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS experiences (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                candidate_id INTEGER,
                organization TEXT NOT NULL,
                position TEXT,
                duration TEXT,
                FOREIGN KEY (candidate_id) REFERENCES candidates(id)
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS relations (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                candidate_id INTEGER,
                relation_type TEXT,
                entity TEXT,
                context TEXT,
                FOREIGN KEY (candidate_id) REFERENCES candidates(id)
            )
        ''')

        conn.commit()
        print(f"Database '{DATABASE_NAME}' and tables created successfully or already exist.")

    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        if conn:
            conn.close()

def update_database_schema():
    """Adds columns for ML classification results to the candidates table."""
    conn = None
    try:
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()

        # Add columns for ML classification results if they don't exist
        try:
            cursor.execute("ALTER TABLE candidates ADD COLUMN job_role_svm TEXT")
        except sqlite3.OperationalError:
            pass # Column already exists
        try:
            cursor.execute("ALTER TABLE candidates ADD COLUMN job_role_bert TEXT")
        except sqlite3.OperationalError:
            pass # Column already exists
        try:
            cursor.execute("ALTER TABLE candidates ADD COLUMN experience_level_svm TEXT")
        except sqlite3.OperationalError:
            pass # Column already exists
        try:
            cursor.execute("ALTER TABLE candidates ADD COLUMN experience_level_bert TEXT")
        except sqlite3.OperationalError:
            pass # Column already exists

        conn.commit()
        print("Database schema update process completed.")

    except sqlite3.Error as e:
        print(f"Database error during schema update: {e}")
    finally:
        if conn:
            conn.close()


# Configuration de l'OCR (Tesseract) - needs to be set if not already
# pytesseract.pytesseract.tesseract_cmd = r'<CHEMIN_VERS_TESSERACT_EXE>' # Replace with your Tesseract path


def get_analysis_from_db(candidate_name):
    """Queries the database for analysis results based on the candidate's name."""
    conn = None
    analysis_data = None
    try:
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()

        # Get candidate ID and classification results
        cursor.execute("SELECT id, name, contact, job_role_svm, job_role_bert, experience_level_svm, experience_level_bert FROM candidates WHERE name = ?", (candidate_name,))
        candidate_info = cursor.fetchone()

        if candidate_info:
            candidate_id, name, contact, job_role_svm, job_role_bert, experience_level_svm, experience_level_bert = candidate_info

            analysis_data = {
                "candidat": {
                    "nom": name,
                    "contact": contact,
                    "education": [],
                    "competences": [],
                    "experiences": [],
                    "job_role_svm": job_role_svm, # Include classification results
                    "job_role_bert": job_role_bert,
                    "experience_level_svm": experience_level_svm,
                    "experience_level_bert": experience_level_bert
                },
                "relations": []
            }

            # Get education
            cursor.execute("SELECT degree FROM education WHERE candidate_id = ?", (candidate_id,))
            education_results = cursor.fetchall()
            analysis_data["candidat"]["education"] = [row[0] for row in education_results]

            # Get skills
            cursor.execute("SELECT skill FROM skills WHERE candidate_id = ?", (candidate_id,))
            skills_results = cursor.fetchall()
            analysis_data["candidat"]["competences"] = [row[0] for row in skills_results]

            # Get experiences
            cursor.execute("SELECT organization, position, duration FROM experiences WHERE candidate_id = ?", (candidate_id,))
            experiences_results = cursor.fetchall()
            analysis_data["candidat"]["experiences"] = [{"organisation": row[0], "poste": row[1], "duree": row[2]} for row in experiences_results]

            # Get relations
            cursor.execute("SELECT relation_type, entity, context FROM relations WHERE candidate_id = ?", (candidate_id,))
            relations_results = cursor.fetchall()
            analysis_data["relations"] = [{"relation": row[0], "entite": row[1], "contexte": row[2]} for row in relations_results]

    except sqlite3.Error as e:
        st.error(f"Database error while retrieving analysis: {e}")
    finally:
        if conn:
            conn.close()

    return analysis_data

# Create tables and update schema if they don't exist when the app starts
create_database_and_tables()
update_database_schema()


st.title("Analyse de CV")

# Section for uploading a new CV
st.header("Analyser un nouveau CV")
uploaded_file = st.file_uploader("Choisissez un fichier PDF ou image (PNG, JPG, JPEG)", type=['pdf', 'png', 'jpg', 'jpeg'])

if uploaded_file is not None:
    with tempfile.NamedTemporaryFile(delete=False, suffix=os.path.splitext(uploaded_file.name)[1]) as tmp_file:
        tmp_file.write(uploaded_file.getvalue())
        tmp_file_path = tmp_file.name

    st.info(f"Fichier téléchargé: {uploaded_file.name}")

    st.text("Analyse en cours...")
    # analyze_cv now saves to DB and includes classification (assuming models are available)
    # Note: The analyze_cv function now requires model objects.
    # For this Streamlit app to work, you would need to load the trained models here
    # or make them globally accessible. This is a simplification for demonstration.
    # Assuming svm_model, tfidf_vectorizer, model (bert_model), tokenizer, label_map
    # are available from previous cells' execution.
    try:
         # Pass the models and label_map to analyze_cv
        analysis_result = analyze_cv(tmp_file_path, svm_model, tfidf_vectorizer, model, tokenizer, label_map)

         # Retrieve data from the database after saving
        if analysis_result and 'candidat' in analysis_result and 'nom' in analysis_result['candidat']:
            candidate_name = analysis_result['candidat']['nom']
            db_analysis_data = get_analysis_from_db(candidate_name)
            if db_analysis_data:
                st.success(f"Analyse terminée et sauvegardée pour {candidate_name}!")
                # Display data retrieved from the database
                st.subheader(f"Résultats de l'analyse pour {candidate_name}")
                # Modify display_results to handle and show classification
                # For now, we'll just display the classification results directly here
                display_results(db_analysis_data) # Reuse the original display function

                st.markdown("#### Classification Results")
                st.write(f"**Job Role (SVM):** {db_analysis_data['candidat'].get('job_role_svm', 'N/A')}")
                st.write(f"**Job Role (BERT):** {db_analysis_data['candidat'].get('job_role_bert', 'N/A')}")
                st.write(f"**Experience Level (SVM):** {db_analysis_data['candidat'].get('experience_level_svm', 'N/A')}")
                st.write(f"**Experience Level (BERT):** {db_analysis_data['candidat'].get('experience_level_bert', 'N/A')}")


            else:
                st.warning(f"Analyse terminée, mais les données pour {candidate_name} n'ont pas pu être récupérées de la base de données.")

        else:
            st.error("Échec de l'analyse du CV. Veuillez vérifier le file content or analysis logic.")

    except NameError as e:
        st.error(f"Error: Required models or variables are not defined. Please ensure all previous cells, including model training, have been executed. Details: {e}")
    except Exception as e:
         st.error(f"An unexpected error occurred during analysis: {e}")


    # Clean up the temporary file
    os.unlink(tmp_file_path)


# Section for viewing previously analyzed CVs
st.header("Voir les analyses précédentes")

# Get list of candidates from the database
conn = None
candidate_names = []
try:
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM candidates ORDER BY name")
    candidate_names = [row[0] for row in cursor.fetchall()]
except sqlite3.Error as e:
    st.error(f"Database error while fetching candidate list: {e}")
finally:
    if conn:
        conn.close()

if candidate_names:
    selected_candidate = st.selectbox("Sélectionnez un candidat:", ["-- Sélectionner --"] + candidate_names)

    if selected_candidate != "-- Sélectionner --":
        st.text(f"Récupération de l'analyse pour {selected_candidate}...")
        previous_analysis_data = get_analysis_from_db(selected_candidate)

        if previous_analysis_data:
            st.subheader(f"Résultats de l'analyse pour {selected_candidate}")
            display_results(previous_analysis_data) # Reuse the display function

            st.markdown("#### Classification Results")
            st.write(f"**Job Role (SVM):** {previous_analysis_data['candidat'].get('job_role_svm', 'N/A')}")
            st.write(f"**Job Role (BERT):** {previous_analysis_data['candidat'].get('job_role_bert', 'N/A')}")
            st.write(f"**Experience Level (SVM):** {previous_analysis_data['candidat'].get('experience_level_svm', 'N/A')}")
            st.write(f"**Experience Level (BERT):** {previous_analysis_data['candidat'].get('experience_level_bert', 'N/A')}")

        else:
            st.warning(f"Aucune analyse trouvée pour {selected_candidate}.")
else:
    st.info("Aucun CV analysé n'a été trouvé dans la base de données.")

2025-07-25 03:02:26.229 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 03:02:26.232 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 03:02:26.235 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 03:02:26.238 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 03:02:26.241 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 03:02:26.244 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 03:02:26.247 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 03:02:26.248 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Database 'cv_analysis.db' and tables created successfully or already exist.
Database schema update process completed.


## Update database schema

### Subtask:
Modify the database schema to store the classification results from the new ML models.

**Reasoning**:
Modify the database schema to add columns for storing the classification results (job role and experience level from both SVM and BERT models). This requires connecting to the SQLite database and executing `ALTER TABLE` statements.

In [29]:
import sqlite3

DATABASE_NAME = 'cv_analysis.db'

def update_database_schema():
    """Adds columns for ML classification results to the candidates table."""
    conn = None
    try:
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()

        # Add columns for ML classification results if they don't exist
        try:
            cursor.execute("ALTER TABLE candidates ADD COLUMN job_role_svm TEXT")
            print("Added 'job_role_svm' column to 'candidates' table.")
        except sqlite3.OperationalError as e:
            if "duplicate column name" in str(e):
                print("'job_role_svm' column already exists.")
            else:
                print(f"Error adding 'job_role_svm' column: {e}")

        try:
            cursor.execute("ALTER TABLE candidates ADD COLUMN job_role_bert TEXT")
            print("Added 'job_role_bert' column to 'candidates' table.")
        except sqlite3.OperationalError as e:
            if "duplicate column name" in str(e):
                print("'job_role_bert' column already exists.")
            else:
                print(f"Error adding 'job_role_bert' column: {e}")

        # Add columns for experience level classification (assuming we add that later)
        try:
            cursor.execute("ALTER TABLE candidates ADD COLUMN experience_level_svm TEXT")
            print("Added 'experience_level_svm' column to 'candidates' table.")
        except sqlite3.OperationalError as e:
            if "duplicate column name" in str(e):
                print("'experience_level_svm' column already exists.")
            else:
                print(f"Error adding 'experience_level_svm' column: {e}")

        try:
            cursor.execute("ALTER TABLE candidates ADD COLUMN experience_level_bert TEXT")
            print("Added 'experience_level_bert' column to 'candidates' table.")
        except sqlite3.OperationalError as e:
            if "duplicate column name" in str(e):
                print("'experience_level_bert' column already exists.")
            else:
                print(f"Error adding 'experience_level_bert' column: {e}")


        conn.commit()
        print("Database schema update process completed.")

    except sqlite3.Error as e:
        print(f"Database error during schema update: {e}")
    finally:
        if conn:
            conn.close()

# Execute the function to update the database schema
update_database_schema()

'job_role_svm' column already exists.
'job_role_bert' column already exists.
'experience_level_svm' column already exists.
'experience_level_bert' column already exists.
Database schema update process completed.


## Refine and improve

### Subtask:
Optimize the performance of the system, improve the UI/UX of the Streamlit app, and add more sophisticated features if needed.

We have reached the refinement stage. Do you have any specific aspects you'd like to improve or refine? This could include:

- Improving the accuracy of the ML models (requires more data and potentially hyperparameter tuning).
- Enhancing the UI/UX of the Streamlit application.
- Adding more advanced entity or relation extraction rules.
- Implementing additional classification tasks (e.g., industry).
- Improving the database interaction or schema.
- Adding error handling or logging.

If you have specific refinements in mind, please let me know! Otherwise, we can proceed to the final step.

## Finish task

### Subtask:
Provide a summary of the implemented features and instructions on how to run the complete system.

Once you are satisfied with the current state of the project, we can proceed to the final step, which is to summarize what we've built and provide instructions on how to run the complete CV analysis system with the Streamlit frontend, database integration, and ML classification.

In [30]:
%%writefile app.py
import streamlit as st
import os
import tempfile
import json
import sqlite3
import re
import spacy
from pdfminer.high_level import extract_text
import pytesseract
from PIL import Image
from spacy.matcher import Matcher
import pandas as pd

# --- NLTK and SpaCy Setup ---
# It's better to ensure these are downloaded before running the app
# For a deployed app, these would be part of the setup/build process.
# !python -m spacy download fr_core_news_sm

# Initialiser le modèle SpaCy pour le français
try:
    nlp = spacy.load("fr_core_news_sm")
except OSError:
    print("Downloading 'fr_core_news_sm' model...")
    os.system("python -m spacy download fr_core_news_sm")
    nlp = spacy.load("fr_core_news_sm")


# --- Database Setup ---
DATABASE_NAME = 'cv_analysis.db'

def create_database_and_tables():
    """Connects to SQLite DB and creates tables if they don't exist."""
    conn = None
    try:
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS candidates (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                contact TEXT,
                job_role_svm TEXT,
                job_role_bert TEXT,
                experience_level_svm TEXT,
                experience_level_bert TEXT
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS education (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                candidate_id INTEGER,
                degree TEXT NOT NULL,
                FOREIGN KEY (candidate_id) REFERENCES candidates(id)
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS skills (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                candidate_id INTEGER,
                skill TEXT NOT NULL,
                FOREIGN KEY (candidate_id) REFERENCES candidates(id)
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS experiences (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                candidate_id INTEGER,
                organization TEXT NOT NULL,
                position TEXT,
                duration TEXT,
                FOREIGN KEY (candidate_id) REFERENCES candidates(id)
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS relations (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                candidate_id INTEGER,
                relation_type TEXT,
                entity TEXT,
                context TEXT,
                FOREIGN KEY (candidate_id) REFERENCES candidates(id)
            )
        ''')
        conn.commit()
    except sqlite3.Error as e:
        st.error(f"Database error during creation: {e}")
    finally:
        if conn:
            conn.close()

def save_analysis_to_db(model):
    """Saves the CV analysis results into the SQLite database."""
    conn = None
    try:
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()
        # Insert candidate information
        c = model['candidat']
        cursor.execute("INSERT INTO candidates (name, contact, job_role_svm, job_role_bert, experience_level_svm, experience_level_bert) VALUES (?, ?, ?, ?, ?, ?)",
                       (c.get('nom'), c.get('contact'), c.get('job_role_svm'), c.get('job_role_bert'), c.get('experience_level_svm'), c.get('experience_level_bert')))
        candidate_id = cursor.lastrowid
        # Insert other details...
        for degree in c['education']:
            cursor.execute("INSERT INTO education (candidate_id, degree) VALUES (?, ?)", (candidate_id, degree))
        for skill in c['competences']:
            cursor.execute("INSERT INTO skills (candidate_id, skill) VALUES (?, ?)", (candidate_id, skill))
        for exp in c['experiences']:
            cursor.execute("INSERT INTO experiences (candidate_id, organization, position, duration) VALUES (?, ?, ?, ?)",
                           (candidate_id, exp.get('organisation'), exp.get('poste'), exp.get('duree')))
        for rel in model['relations']:
            cursor.execute("INSERT INTO relations (candidate_id, relation_type, entity, context) VALUES (?, ?, ?, ?)",
                           (candidate_id, rel.get('relation'), rel.get('entite'), rel.get('contexte')))
        conn.commit()
    except sqlite3.Error as e:
        st.error(f"Database error while saving analysis: {e}")
        if conn:
            conn.rollback()
    finally:
        if conn:
            conn.close()

# --- Analysis Functions ---
def extract_data(file_path):
    try:
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            img = Image.open(file_path)
            text = pytesseract.image_to_string(img, lang='fra')
        elif file_path.lower().endswith('.pdf'):
            text = extract_text(file_path)
        else:
            raise ValueError("Format de fichier non supporté")
        return text
    except Exception as e:
        st.error(f"Erreur d'extraction: {str(e)}")
        return ""

def clean_data(text):
    text = re.sub(r'[^\w\s.,;:!?À-ÿ-]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.lower().strip()

def extract_entities(text):
    doc = nlp(text)
    entities = {"PERSON": [], "ORG": [], "DATE": [], "LOC": [], "DIPLOME": [], "COMPETENCE": []}
    for ent in doc.ents:
        if ent.label_ in entities and ent.text not in entities[ent.label_]:
            entities[ent.label_].append(ent.text)
    diplome_patterns = [
        [{"LOWER": {"IN": ["bac", "bts", "dut", "licence", "master", "doctorat"]}}],
        [{"LOWER": "bachelor"}, {"LOWER": "of"}, {"LOWER": "science"}],
        [{"LOWER": "ingénieur"}]]
    matcher = Matcher(nlp.vocab)
    matcher.add("DIPLOME", diplome_patterns)
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        if span.text not in entities["DIPLOME"]:
            entities["DIPLOME"].append(span.text)
    return entities

def extract_relations(entities, text):
    relations = []
    doc = nlp(text)
    for sentence in doc.sents:
        if "expérience" in sentence.text:
            for competence in entities["COMPETENCE"]:
                if competence in sentence.text:
                    relations.append({"relation": "a_experience_avec", "entite": competence, "contexte": sentence.text.strip()})
    return relations

def build_model(entities, relations):
    model = {
        "candidat": {
            "nom": entities["PERSON"][0] if entities["PERSON"] else "Inconnu",
            "contact": "",
            "education": entities["DIPLOME"],
            "competences": entities["COMPETENCE"],
            "experiences": []
        },
        "relations": relations
    }
    for org in entities["ORG"]:
        model["candidat"]["experiences"].append({"organisation": org, "poste": "À déterminer", "duree": ""})
    return model

def classify_cv(text):
    # Placeholder for actual classification logic
    # In a real app, you would load your trained models here and use them
    return {
        "job_role_svm": "Not Implemented",
        "job_role_bert": "Not Implemented",
        "experience_level_svm": "Not Implemented",
        "experience_level_bert": "Not Implemented"
    }

def analyze_cv(file_path):
    st.text(f"Début de l'analyse du fichier: {os.path.basename(file_path)}")
    raw_text = extract_data(file_path)
    if not raw_text: return None
    cleaned_text = clean_data(raw_text)
    entities = extract_entities(cleaned_text)
    relations = extract_relations(entities, cleaned_text)
    model_data = build_model(entities, relations)
    classification_results = classify_cv(cleaned_text)
    model_data['candidat'].update(classification_results)
    save_analysis_to_db(model_data)
    return model_data

# --- Streamlit UI ---
def display_results(model):
    st.subheader(f"Résultats de l'analyse pour {model['candidat']['nom']}")
    st.markdown("#### Informations Candidat")
    st.write(f"**Nom:** {model['candidat'].get('nom', 'N/A')}")
    st.markdown("#### Formation")
    st.write(model['candidat']['education'] if model['candidat']['education'] else "Aucune information trouvée.")
    st.markdown("#### Compétences")
    st.write(model['candidat']['competences'] if model['candidat']['competences'] else "Aucune compétence trouvée.")
    st.markdown("#### Expériences Professionnelles")
    st.write(model['candidat']['experiences'] if model['candidat']['experiences'] else "Aucune expérience trouvée.")
    st.markdown("#### Classification Results")
    st.write(f"**Job Role (SVM):** {model['candidat'].get('job_role_svm', 'N/A')}")
    st.write(f"**Job Role (BERT):** {model['candidat'].get('job_role_bert', 'N/A')}")

def get_analysis_from_db(candidate_name):
    conn = None
    try:
        conn = sqlite3.connect(DATABASE_NAME)
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM candidates WHERE name = ?", (candidate_name,))
        candidate_info = cursor.fetchone()
        if candidate_info:
            # Reconstruct the model from DB data (simplified)
            # This part needs to be more robust in a real application
            model = {'candidat': {'nom': candidate_info[1]}} # Basic reconstruction
            return model
        return None
    except sqlite3.Error as e:
        st.error(f"Database error while retrieving analysis: {e}")
    finally:
        if conn: conn.close()
    return None


# --- Main App ---
create_database_and_tables()
st.title("Analyse de CV")

st.header("Analyser un nouveau CV")
uploaded_file = st.file_uploader("Choisissez un fichier PDF ou image", type=['pdf', 'png', 'jpg', 'jpeg'])

if uploaded_file is not None:
    with tempfile.NamedTemporaryFile(delete=False, suffix=os.path.splitext(uploaded_file.name)[1]) as tmp_file:
        tmp_file.write(uploaded_file.getvalue())
        tmp_file_path = tmp_file.name

    analysis_result = analyze_cv(tmp_file_path)
    os.unlink(tmp_file_path)

    if analysis_result:
        st.success("Analyse terminée avec succès!")
        display_results(analysis_result)

st.header("Voir les analyses précédentes")
conn = None
try:
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM candidates ORDER BY name")
    candidate_names = [row[0] for row in cursor.fetchall()]
    selected_candidate = st.selectbox("Sélectionnez un candidat:", ["-- Sélectionner --"] + candidate_names)
    if selected_candidate != "-- Sélectionner --":
        db_analysis_data = get_analysis_from_db(selected_candidate)
        if db_analysis_data:
            display_results(db_analysis_data)
        else:
            st.warning("Aucune analyse trouvée.")
except sqlite3.Error as e:
    st.error(f"Database error fetching candidates: {e}")
finally:
    if conn: conn.close()

Overwriting app.py


In [ ]:
!streamlit run app.py